## 개요 

* 2024년 12회 산업통상자원부 공공데이터 활용 아이디어 공모전 참가해보기로 함
  * https://datacontest.kr/apply/applyAdd/3
* 단순 API로딩이 아니라, 각자 바로 데이터를 로딩할 수 있는 매체에 대한 고민 후 구현
  * 코딩이 익숙하지 않은 팀원이 쉽게 이용할 수 있도록하고, readme에 바로 사용할 수 있게 샘플코드 제공
  * github를 활용해 pandas에서 바로 로딩할 수 있도록 구현

## 내용정리

### 도입목적
* 본격적으로 모델 학습을 하기 전, 데이터 이용 편의를 증진하고자 함
* 팀원들이 R이나 통계분석에는 익숙하나 파이썬 코딩에는 익숙하지않아, 최대한 모델링에 집중하도록 지원
  * 하나의 repository에서 원하는 데이터를 한번에 확인 가능
  * 업데이트 일자를 표기하여 얼마나 최신 데이터인지 확인 가능
* 데이터를 하나의 페이지에서 통합관리(공공데이터 홈페이지 접속 등 불필요)
* 개인서버(NAS)에서 매일 특정시간 구동하여 별도의 수작업없이 자동으로 최신화

### 구동방식
* 공공데이터 리스트와 API키가 저장된 json파일 로딩
* 지정된 공공데이터를 다운로드하고 csv파일로 저장
* 바로 로딩하기위한 파일 주소생성, 업데이트 날짜 저장
* README 파일에 파일주소와 업데이트 날짜 등 업데이트
* git_push함수로 github repository에 자동업로드

### github reposiroty주소
[https://github.com/KR9268/db_datagokr](https://github.com/KR9268/db_datagokr)

### 샘플코드(패키지 및 함수)

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime
import subprocess
import os
import time

def json_load(json_path, encoding='utf-8'):
    with open(json_path, 'r', encoding=encoding) as file:
        json_data = json.load(file)
    return json_data

def request_and_to_json(url):
    response = requests.get(url)
    json_ob = json.loads(response.text)
    return json_ob

def chk_json_status_of_data_go_kr(json_obj):
    other_data = ['currentCount', 'matchCount', 'page', 'perPage', 'totalCount']
    result_dict = dict()
    
    for each_column in other_data:
        result_dict[each_column] = json_obj[each_column]  
    return result_dict 

def download_from_data_go_kr_with_json(url):
    json_ob = request_and_to_json(url)

    json_status = chk_json_status_of_data_go_kr(json_ob)
    if json_status['currentCount'] < json_status['totalCount']:
        url = url.replace('perPage=1',f'perPage={json_status["totalCount"]}')
        json_ob = request_and_to_json(url)

    return json_ob

def update_readme(new_content_list):
    # Open the README.md file in read mode
    with open('README.md', 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Find the index of the line that starts with '* 데이터 현황'
    index = next((i for i, line in enumerate(lines) if line.startswith('* 데이터 현황')), None)

    # If the line is found, remove the following lines and insert new content
    if index is not None:
        lines = lines[:index+1] # Remove the following lines
        #lines.extend(new_content) # Insert new content
        lines.extend(new_content_list) # Insert new content

    # Open the README.md file in write mode and write the updated content
    with open('README.md', 'w', encoding='utf-8') as file:
        file.writelines(lines)

def git_push():
    # Get a list of all .csv files in the current directory
    csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]

    # Stage all .csv files
    for file in csv_files:
        subprocess.call(['git', 'add', file])

    subprocess.call(['git', 'add', 'README.md'])
    # Commit the changes with a message
    subprocess.call(['git', 'commit', '-m', 'Automatic commit'])

    # Push the changes to the remote repository
    subprocess.call(['git', 'push'])

### 샘플코드(메인코드)

In [ ]:
# json load
serviceKey = json_load('option.json')['serviceKey']
db_list = json_load('db_list.json', 'cp949')

In [ ]:
# main

# 작업하기
txt_for_readme = ['\n']
for each in db_list:
    # 다운로드
    url = f"{each['base_url']}{each['address_get']}?page=1&perPage=1&serviceKey={serviceKey}"
    json_data = download_from_data_go_kr_with_json(url)
    result_data = chk_json_status_of_data_go_kr(json_data)
    
    # 저장
    if result_data['currentCount'] == result_data['totalCount']:
        pd.json_normalize(json_data['data']).to_csv(f"{each['file_name_to']}.csv",encoding='cp949', index=False)

    # 파일주소 및 이름, 업데이트시간 저장
    owner = 'KR9268'
    repo = 'db_datagokr'
    branch = 'main'
    file_path = f"{each['file_name_to']}.csv"

    url = f"https://raw.githubusercontent.com/{owner}/{repo}/{branch}/{file_path}"

    txt_for_readme.append(f"  *  {datetime.strftime(datetime.now(),'%Y-%m-%d')}업데이트 : {each['name']}\n{url}\n")
    time.sleep(1)

# 업데이트 내역과 파일 git push
update_readme(txt_for_readme)
git_push()